In [32]:
import pandas as pd
import re
import tensorflow as tf
import pickle

In [33]:
df = pd.read_pickle('data/tokenized_dataset.pkl')

In [34]:
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc,tokenized_abc,note_seq,dur_seq
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:...","[G_0.500, c_0.500, B_0.500, A_0.500, G_0.500, ...","[G, c, B, A, G, E, D, C, A, B, C, A, B, C, E, ...","[0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5..."
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...,"[G_0.500, E_0.500, E_0.500, E_1.000, D_0.500, ...","[G, E, E, E, D, E, D, C, G, E, E, E, F, G, A, ...","[0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...,"[F_0.500, D_0.500, D_0.500, D_0.500, D_0.500, ...","[F, D, D, D, D, C, D, C, B, F, D, D, D, E, F, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...,"[C_0.500, D_0.500, E_0.500, C_0.500, F_0.500, ...","[C, D, E, C, F, D, E, C, A, A, D, C, B, G, G, ...","[0.75, 0.25, 0.75, 0.25, 0.75, 0.25, 0.75, 0.2..."
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,...","[G_0.500, B_0.500, C_0.500, C_1.000, E_0.500, ...","[G, B, C, C, E, G, F, D, E, C, G, A, D, C, B, ...","[0.5, 0.5, 0.5, 1.0, 0.75, 0.25, 0.75, 0.25, 0..."


In [35]:
all_notes = sorted({n for seq in df.note_seq for n in seq})
note_lookup = tf.keras.layers.StringLookup(
    vocabulary=all_notes,
    mask_token=None,    # 0 will be reserved for padding
    oov_token="[UNK]"
)


I0000 00:00:1747887701.675152   15537 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9558 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


In [36]:
df['note_ids'] = df.note_seq.apply(lambda seq: note_lookup(seq).numpy().tolist())
# dur_seq stays as a list of floats

In [37]:
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc,tokenized_abc,note_seq,dur_seq,note_ids
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:...","[G_0.500, c_0.500, B_0.500, A_0.500, G_0.500, ...","[G, c, B, A, G, E, D, C, A, B, C, A, B, C, E, ...","[0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5...","[19, 58, 14, 13, 19, 17, 16, 15, 13, 14, 15, 1..."
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...,"[G_0.500, E_0.500, E_0.500, E_1.000, D_0.500, ...","[G, E, E, E, D, E, D, C, G, E, E, E, F, G, A, ...","[0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, ...","[19, 17, 17, 17, 16, 17, 16, 15, 19, 17, 17, 1..."
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...,"[F_0.500, D_0.500, D_0.500, D_0.500, D_0.500, ...","[F, D, D, D, D, C, D, C, B, F, D, D, D, E, F, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...","[18, 16, 16, 16, 16, 15, 16, 15, 14, 18, 16, 1..."
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...,"[C_0.500, D_0.500, E_0.500, C_0.500, F_0.500, ...","[C, D, E, C, F, D, E, C, A, A, D, C, B, G, G, ...","[0.75, 0.25, 0.75, 0.25, 0.75, 0.25, 0.75, 0.2...","[15, 16, 17, 15, 18, 16, 17, 15, 13, 13, 16, 1..."
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,...","[G_0.500, B_0.500, C_0.500, C_1.000, E_0.500, ...","[G, B, C, C, E, G, F, D, E, C, G, A, D, C, B, ...","[0.5, 0.5, 0.5, 1.0, 0.75, 0.25, 0.75, 0.25, 0...","[19, 14, 15, 15, 17, 19, 18, 16, 17, 15, 19, 1..."


In [45]:
# In Notebook A, after you’ve built note_lookup…
vocab_list = note_lookup.get_vocabulary()

import pickle
with open("note_vocab.pkl", "wb") as f:
    pickle.dump(vocab_list, f)


In [41]:
# time to remove tunes with only one setting - at least for training triplet loss
counts = df.tune_id.value_counts()
good_ids = counts[counts > 1].index
df = df[df.tune_id.isin(good_ids)]


In [ ]:
df.to_pickle

In [ ]:
tf.data.Dataset.save(ds, 'data/ts_dataset_triplet')